A very simple notebook containing the following steps:
- load the train set (parquet format)
- reduce the memory usage
- set the LGBM parameters optimised with Optuna
- fit and submit

# 1. Utils

In [ ]:
def reduce_mem_usage(df):
#     https://www.kaggle.com/code/edwardakalarrywelch/datatable-memory-reduction
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024**2
    
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
     
    return df

# 2. Load Libraries and Data

In [ ]:
import numpy as np
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from lightgbm import LGBMRegressor

import ubiquant

df = reduce_mem_usage(pd.read_parquet('../input/ubiquant-parquet/train_low_mem.parquet'))

# 3. Modelling

In [ ]:
features = ['investment_id'] + ['f_' + str(i) for i in range(100)]

# Params from https://www.kaggle.com/code/valleyzw/ubiquant-lgbm-optimization
params = {'lambda_l1': 0.03627602394442367, 'lambda_l2': 0.43523855951142926, 'num_leaves': 114, 'feature_fraction': 0.9505625064462319,
     'bagging_fraction': 0.9785558707339647, 'bagging_freq': 7, 'max_depth': -1, 'max_bin': 501, 'min_data_in_leaf': 374}

model = LGBMRegressor(**params)
model.fit(df[features].loc[df.time_id >= 600], df['target'].loc[df.time_id >= 600])

In [ ]:
plot_importance(model, height=1, figsize=(20, 20))

# 4. Submission

In [ ]:
env = ubiquant.make_env()   # initialize the environment
iter_test = env.iter_test()    # an iterator which loops over the test set and sample submission

for (test_df, sample_prediction_df) in iter_test:
    print(test_df.head())
    sample_prediction_df['target'] =  model.predict(test_df[features])  # make your predictions here
    env.predict(sample_prediction_df)   # register your predictions